In [1]:
%matplotlib inline
import seaborn
import quandl
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.formula.api as sm
token="Us3wFmXGgAj_1cUtHAAR"

/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
/Users/mrefermat/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
tsmom_model.py:162: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
tsmom_model.py:48: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(min_periods=100,window=250,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:48: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
tsmom_model.py:51: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future ve

In [43]:
liquidity_factors=get_all_factors()

In [64]:
final_res=pd.DataFrame()
r2=pd.Series()
rmse=pd.Series()
alpha={}
tstat={}
for sec in liquidity_factors.columns:
    ind=liquidity_factors.dropna().index
    reges=get_aqr_factors('All').loc[ind]
    reges['Market Return']=cleansed.resample(rule='m',how='last').pct_change().mean(axis=1).dropna()
    reges['const']=1
    #reges['TERM']=quandl.get('FRED/THREEFYTP10',authtoken=token).Value.resample(rule='m',how='last')
    res=sm.OLS(liquidity_factors[sec].loc[ind],reges).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    final_res[sec]=res.params
    final_res[sec+'Tstats']=res.tvalues
    r2[sec]=res.rsquared
    rmse[sec]=math.sqrt(res.mse_model)
    


/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  if __name__ == '__main__':


In [65]:
final_res.T.round(4)[['const', u'Market Return','VAL', u'MOM' ]]

,const,Market Return,VAL,MOM
Amihud Annual,0.0014,0.0201,0.0277,-0.0475
Amihud AnnualTstats,1.1834,0.3542,0.3726,-0.6021
FHT Annual,0.0016,0.0874,-0.2393,-0.2448
FHT AnnualTstats,1.2629,1.6506,-2.5278,-3.7046
Amihud Monthly,0.0010,-0.1184,-0.0557,-0.0336
Amihud MonthlyTstats,1.6826,-4.3743,-1.2596,-0.9500
FHT Monthly,-0.0006,0.1289,-0.1374,-0.1354
FHT MonthlyTstats,-0.4525,2.3288,-1.3029,-1.8266


In [66]:
print final_res.T.round(2)[['const', u'Market Return','VAL', u'MOM' ]].to_latex()

\begin{tabular}{lrrrr}
\toprule
{} &  const &  Market Return &   VAL &   MOM \\
\midrule
Amihud Annual        &   0.00 &           0.02 &  0.03 & -0.05 \\
Amihud AnnualTstats  &   1.18 &           0.35 &  0.37 & -0.60 \\
FHT Annual           &   0.00 &           0.09 & -0.24 & -0.24 \\
FHT AnnualTstats     &   1.26 &           1.65 & -2.53 & -3.70 \\
Amihud Monthly       &   0.00 &          -0.12 & -0.06 & -0.03 \\
Amihud MonthlyTstats &   1.68 &          -4.37 & -1.26 & -0.95 \\
FHT Monthly          &  -0.00 &           0.13 & -0.14 & -0.14 \\
FHT MonthlyTstats    &  -0.45 &           2.33 & -1.30 & -1.83 \\
\bottomrule
\end{tabular}



In [53]:
(r2*100).round(2)

Amihud Annual      0.96
FHT Annual         8.99
Amihud Monthly    14.77
FHT Monthly        5.65
dtype: float64

In [70]:
rmse.round(3)

Amihud Annual     0.013
FHT Annual        0.042
Amihud Monthly    0.023
FHT Monthly       0.035
dtype: float64